In [1]:
# Import the package
import bjontegaard as bd
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os
import pandas as pd
import matplotlib.pyplot as plt
from sewar.full_ref import msssim

In [2]:
## Define class that will be used for data laoding and comparison
class ImageDataset:
    def __init__(self, directory='./output_images_sota/'):
        """
        Initialize the dataset with the path to the directory containing images
        """
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory)]
        self.file_names = [f for f in os.listdir(directory)]

    def __len__(self):
        """
        Return the number of images in the dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, index):
        """
        Load and return an image at the specified index.
        """
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        return np.array(image)

    def get_filename(self,index):
        """
        Return the filenames of the images.
        """
        return self.file_names[index]
    
    def get_paths(self):
        """
        Return the file paths of the images.
        """
        return self.image_paths   
            
class CompressionMetrics:
    def __init__(self, test: ImageDataset, anchor: ImageDataset):
        """
        Initialize the CompressionMetrics class with instances of ImageDataset for
        test and anchor datasets.
        """
        self.test = test
        self.anchor = anchor
    
    def calculate_bit_rate(self, image_path):
        """
        Helper function to calculate the bit rate of the image. Return the bit rate
        """
        # Get the file size in bytes and convert to bits
        file_size_bytes = os.path.getsize(image_path)
        file_size_bits = file_size_bytes * 8  # Convert bytes to bits

        # Load the image to get its dimensions
        image = cv2.imread(image_path)
        num_pixels = image.shape[0] * image.shape[1]  # width x height

        # Calculate the bit rate (bits per pixel)
        bit_rate = file_size_bits / num_pixels

        return bit_rate
    
    def calculate_group_bit_rate(self):
        """
        Return the bit rates of images using helper function calculate_bit_rate()
        """
        bit_rates = [self.calculate_bit_rate(i) for i in self.test.image_paths]
        return bit_rates
    
    def calculate_psnr(self,img1, img2):
        """
        Helper function to calculate the psnr of the image after compression. Return the psnr value.
        """
        mse = np.mean((img1 - img2) ** 2)
        if mse == 0:
            return float('inf')
        max_pixel = 255.0
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr
    
    def calculate_group_psnr(self):
        """
        Return the psnr of images using helper function calculate_psnr().
        """
        if not self.test.__len__() == self.anchor.__len__():
            raise ValueError(f"Test and anchor dataset are not of the same length.")       
        psnrs = [self.calculate_psnr(cv2.imread(self.anchor.image_paths[i]), cv2.imread(self.test.image_paths[i])) for i in range(self.test.__len__())]
        return psnrs
    
    def caculate_group_compression_rate(self):
        """
        Return the the compression rate of images
        """
        if not self.test.__len__() == self.anchor.__len__():
            raise ValueError(f"Test and anchor dataset are not of the same length.")       
        psnrs = [self.calculate_psnr(cv2.imread(self.anchor.image_paths[i]), cv2.imread(self.test.image_paths[i])) for i in range(self.test.__len__())]        
        cr = [os.path.getsize(self.test.image_paths[i])/ os.path.getsize(self.anchor.image_paths[i]) for i in range(self.test.__len__())]
        return cr
    
    
    def calculate_group_mmsim(self):
        """
        Return the mmssim of images.
        """
        if not self.test.__len__() == self.anchor.__len__():
            raise ValueError(f"Test and anchor dataset are not of the same length.")       
        msssims = [msssim(cv2.imread(self.anchor.image_paths[i]), cv2.imread(self.test.image_paths[i])) for i in range(self.test.__len__())]
        return msssims


In [3]:
def compare_methods_new(rate_anchor,
                    dist_anchor,
                    rate_test,
                    dist_test,
                    require_matching_points=True,
                    rate_label='rate',
                    distortion_label='PSNR',
                    figure_label=None,
                    filepath=None):
    """
    Plots a comparison of the internal behaviour of the different interpolation methods for BD calculations.

    :param rate_anchor: rates of reference codec
    :param dist_anchor: distortion metrics of reference codec
    :param rate_test: rates of investigated codec
    :param dist_test: distortion metrics of investigated codec
    :param require_matching_points: whether to require an equal number of rate-distortion points for anchor and test.
    (default: True)
    :param rate_label: Rate metric label (x-axis)
    :param distortion_label: Distortion metric label (y-axis)
    :param figure_label: Figure label (title)
    :param filepath: if filepath is given, final plot is stored to the given file
    :raises ValueError: if number of points for rate and distortion metric do not match
    :raises ValueError: if `require_matching_points == True` and number of rate-distortion points for anchor and test
    do not match
    :raises ValueError: if interpolation method is not valid
    """
    rate_anchor = np.asarray(rate_anchor)
    dist_anchor = np.asarray(dist_anchor)
    rate_test = np.asarray(rate_test)
    dist_test = np.asarray(dist_test)

    dists1 = np.linspace(dist_anchor.min(), dist_anchor.max(), num=10, endpoint=True)
    dists2 = np.linspace(dist_test.min(), dist_test.max(), num=10, endpoint=True)

    # Plot interpolation curves for each method
    methods = {
        'cubic': 'Cubic interpolation (non-piece-wise)',
        'pchip': 'Piece-wise cubic interpolation',
        'akima': 'BD Calculation with Akima Interpolation'
    }
    fig, axs = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle(figure_label)
    for ax, (method, label) in zip(axs.flat, methods.items()):
        bd_rate, interp1, interp2 = bd.bd_rate(rate_anchor, dist_anchor, rate_test, dist_test,
                                               method=method,
                                               require_matching_points=require_matching_points,
                                               interpolators=True)
        bd_psnr = bd.bd_psnr(rate_anchor, dist_anchor, rate_test, dist_test, method=method, require_matching_points=require_matching_points)

        # Plot rate1 and dist1
        rates1 = interp1(dists1)
        ax.plot(np.log10(rate_anchor), dist_anchor, '-o', color='tab:blue', label='anchor')
        ax.plot(rates1, dists1, '--', color='tab:blue')

        # Plot rate2 and dist1
        rates2 = interp2(dists2)
        ax.plot(np.log10(rate_test), dist_test, '-o', color='tab:orange', label='test')
        ax.plot(rates2, dists2, '--', color='tab:orange')

        # Set axis properties
        ax.set_title(label)
        ax.set_xlabel('{}({})'.format(np.log10.__name__, rate_label))
        ax.set_ylabel(distortion_label)
        ax.grid()
        ax.legend()

        # Add bd metrics table
        cell_text = [
            ["{:.10f} %".format(bd_rate)],
            ["{:.10f} dB".format(bd_psnr)]
        ]
        ax.table(cellText=cell_text, rowLabels=["BD-Rate", "BD-PSNR"],
                 colWidths=[0.3, 0.1], loc="lower right", zorder=10)

    # Remove unused axes
    if len(axs.flat) > len(methods):
        for ax in axs.flat[len(methods):]:
            ax.axis('off')

    # Save if filepath is given
    if filepath is not None:
        fig.savefig(filepath, dpi=fig.dpi)

    plt.show()

In [5]:

## Read in data using ImageDataSet class defined
original = ImageDataset('./converted/input_images/')
rotated_original = ImageDataset('./converted/rotated_dataset/')
sota = ImageDataset('./output_images_sota/')
pca = ImageDataset('./output_images_pca/')
svd = ImageDataset('./output_images_svd/')
dct = ImageDataset('./ouput_images_dct/')
wht = ImageDataset('./ouput_images_wht/')
## Get corresponding metrics of eacg method
metric_sota = CompressionMetrics(sota, original)
metric_pca = CompressionMetrics(pca, rotated_original)
metric_svd = CompressionMetrics(svd, rotated_original)
metric_dct = CompressionMetrics(dct, original)
metric_wht = CompressionMetrics(wht, original)

In [6]:
comparison_list = [metric_pca,metric_svd,metric_dct,metric_wht, metric_sota]
#cr_list = [np.mean(i.caculate_group_compression_rate()) for i in comparison_list]
psnrs = [i.calculate_group_psnr() for i in comparison_list]
psnr_list = [np.mean(p) for p in psnrs]

msssims = [i.calculate_group_mmsim() for i in comparison_list]
msssim_list = [np.mean(p) for p in msssims]
## turn it from complex type into float type
msssim_list = [float(m.real) for m in msssim_list]

## For bpp, it is read directly from the output log from each methods
# bit_list = [np.mean(i.calculate_group_bit_rate()) for i in comparison_list]

## For dct and wht
bitrate_dct = [
    12.122884114583334, 11.717244466145834, 9.500874837239584, 11.1080322265625,
    12.97540283203125, 11.970967610677084, 10.149210611979166, 14.0152587890625,
    11.06463623046875, 11.16644287109375, 11.502421061197916, 10.39691162109375,
    14.461344401041666, 12.39215087890625, 10.681355794270834, 10.513285319010416,
    10.749593098958334, 13.506876627604166, 11.784952799479166, 11.725240071614584,
    12.17486572265625, 12.496236165364584, 10.247049967447916, 12.24713134765625
]

bitrate_wht = [
    5.12530517578125, 3.5201416015625, 3.1517333984375, 3.8216552734375,
    6.318461100260417, 4.441304524739583, 3.9278971354166665, 5.796875,
    3.0187174479166665, 3.3409830729166665, 4.2320556640625, 3.21337890625,
    6.0504150390625, 5.520975748697917, 3.3631795247395835, 3.6353556315104165,
    3.60479736328125, 4.87548828125, 3.8236083984375, 3.0378621419270835,
    3.91546630859375, 4.059163411458333, 3.0528157552083335, 4.85302734375
] 
mean_bitrate_dct = sum(bitrate_dct) / len(bitrate_dct)
mean_bitrate_wht = sum(bitrate_wht) / len(bitrate_wht)

## For svd (rank 30) 
mean_bitrate_svd = 4.691162109375

## For SOTA, we get this data directly from the log file
sota_df = pd.read_csv("./sota_result.csv")
psnr_list[3] = np.mean(sota_df['psnr'])
msssim_list[3] = np.mean(sota_df['ms_ssim'])
mean_bitrate_sota = np.mean(sota_df['bpp'])

## the compression rate of PCA does not meet the requirement, so we did not put it here
bitrate_list = ["Not applicable", mean_bitrate_svd, mean_bitrate_dct, mean_bitrate_wht,mean_bitrate_sota]

In [10]:
# Method names
methods = ['PCA', 'SVD', 'DCT', 'WHT','MLIC++']

# Create a dictionary where keys are column names and values are your lists
data = {
    'PSNR[dB]': psnr_list,
    "MS-SSIM[dB]": msssim_list,
    'Bit-rate[bpp]': bitrate_list
}

# Create the DataFrame
df = pd.DataFrame(data, index=methods)

In [11]:
df

,PSNR[dB],MS-SSIM[dB],Bit-rate[bpp]
PCA,35.053120,0.869777,Not applicable
SVD,31.180132,0.870445,4.691162
DCT,45.266768,0.981600,11.694599
WHT,29.135300,0.927178,4.154194
MLIC++,32.303320,0.930553,0.108297


In [12]:
## without PCA result
df[:][1:]

,PSNR[dB],MS-SSIM[dB],Bit-rate[bpp]
SVD,31.180132,0.870445,4.691162
DCT,45.266768,0.981600,11.694599
WHT,29.135300,0.927178,4.154194
MLIC++,32.303320,0.930553,0.108297
